# Runtime Resources Check
Check that we are allocated a heavy-duty GPU. This will be necessary for running OpenPose.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Apr 10 16:16:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


# Software Installation

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp39-cp39-linux_x86_64.whl size=44089 sha256=7973dd7464f1ad8fd2b85ac8893aa76fff735d4ef1815a5cd1bffd5d6e9349f5
  Stored in directory: /root/.cache/pip/wheels/68/be/8f/b6c454cd264e0b349b47f8ee00755511f277618af9e5dae20d
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flax 0.6.8 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatible.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, shutil
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


## OpenPose

In [ ]:
# Openpose バージョン指定タグ
# Openpose version tag
ver_openpose = "v1.7.0"
! echo $ver_openpose

# CMakeが古いとOpenpose（CUDA10)が失敗するので、バージョンを確認します。（BugfixはCMake 3.12.3）
# If CMake is old, Openpose build fails, so download the latest version
# https://developercommunity.visualstudio.com/content/problem/354325/cmake-project-cannot-detect-cuda-10.html
! cmake --version

# 2021.03.03時点の最新CMakeを再ビルド（15分くらい）
# Rebuild the latest CMake as of 2021.03.03 (about 15 minutes)
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install

# ライブラリインストール
# Install library

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

# Openposeのコードをclone
# Clone Openpose
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 

# build用ディレクトリを作成
# Create build directory
! cd openpose && mkdir build && cd build

# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# 上記インストール手順のシナリオ１でインストール実行
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
! cd openpose/build && cmake .. 

# COCOモデルのDLオプション付き(「# ! cd」の部分を「! cd」に変更してください。)
# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON

# MPIモデルのDLオプション付き（同上）
# If you want to download the MPI model as well, execute the following command.
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

# Openposeのビルド（15分くらい）
# Openpose Building
! cd openpose/build && make -j`nproc`
# outputフォルダ作成
! cd openpose && mkdir output

# デモ動画を解析
# 出力された解析結果は、Colab画面の左メニューのフォルダアイコンから「openpose/output」以下に配置されます。
# Run and check the sample
# The output analysis result is placed under "openpose/output" from the folder icon on the left menu of the Colab screen.
#! cd openpose && ./build/examples/openpose/openpose.bin --video ../drive/MyDrive/NNextPitch/example_vid.mp4 --display 0  --write_video ../drive/MyDrive/NNextPitch/example_vid_openpose.mp4

Streaming output truncated to the last 5000 lines.
-- Installing: /usr/local/share/cmake-3.19/Help/generator/Visual Studio 9 2008.rst
-- Installing: /usr/local/share/cmake-3.19/Help/generator/Borland Makefiles.rst
-- Installing: /usr/local/share/cmake-3.19/Help/generator/Visual Studio 15 2017.rst
-- Installing: /usr/local/share/cmake-3.19/Help/generator/Visual Studio 10 2010.rst
-- Installing: /usr/local/share/cmake-3.19/Help/generator/Kate.rst
-- Installing: /usr/local/share/cmake-3.19/Help/generator/MSYS Makefiles.rst
-- Installing: /usr/local/share/cmake-3.19/Help/generator/Green Hills MULTI.rst
-- Installing: /usr/local/share/cmake-3.19/Help/policy
-- Installing: /usr/local/share/cmake-3.19/Help/policy/CMP0102.rst
-- Installing: /usr/local/share/cmake-3.19/Help/policy/CMP0076.rst
-- Installing: /usr/local/share/cmake-3.19/Help/policy/CMP0044.rst
-- Installing: /usr/local/share/cmake-3.19/Help/policy/CMP0006.rst
-- Installing: /usr/local/share/cmake-3.19/Help/policy/CMP0030.rst
-- I

In [ ]:
# Set root path and file names
root_path = "drive/MyDrive/NNextPitch"

# Pipeline Setup

## Import Detectron2 Model
This model has been trained to detect pitchers

In [ ]:
# Set inference parameters
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = os.path.join(root_path, "d2-output/model_final.pth") # Path to the weights of the training model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.97 # Set a custom testing threshold (probability threshold of calling object a pitcher)
cfg.TEST.DETECTIONS_PER_IMAGE = 1 # Maximum number of detections per image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # Only has one class (pitcher)

# Set model
d2_predictor = DefaultPredictor(cfg)

[04/10 16:46:35 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from drive/MyDrive/NNextPitch/d2-output/model_final.pth ...


## Import Pitcher Video URL JSON

In [ ]:
import json

with open(os.path.join(root_path, "data/video-url-json/NNextPitchURLs.json")) as f:
    pitcher_info = json.load(f)

## Pipeline Functions

In [ ]:
import requests
import cv2
import os, shutil


def download_mp4(video_link, output_dir):

  # Get video from url
  chunk_size = 256
  r = requests.get(video_link, stream=True)

  # Make output directory if it doesn't exist
  os.makedirs(output_dir, exist_ok=True)

  # Download mp4
  output_name = video_link.split(".com/")[1]
  with open(f"{output_dir}/{output_name}", "wb") as f:
    for chunk in r.iter_content(chunk_size=chunk_size):
      f.write(chunk)


def mp4_to_image(mp4, output_dir, skip_n_frames=5):

  # Make output directory if it doesn't exist
  os.makedirs(output_dir, exist_ok=True)
    
  # Output each individual frame as a jpg
  vidcap = cv2.VideoCapture(mp4)
  count = 0
  i = 0
  while vidcap.isOpened():
    success, image = vidcap.read()
    if success:
      if count % skip_n_frames == 0:
        output_file = f"{output_dir}/{mp4.split('/')[-1].split('.mp4')[0]}-{i}.jpg"
        cv2.imwrite(output_file, image)
        i += 1
      count += 1
    else:
      break
  cv2.destroyAllWindows()
  vidcap.release()


def blur_image(image_path, output_dir):

  # Import image
  image = cv2.imread(image_path)
  image_name = image_path.split("/")[-1].split(".jpg")[0]
  out_image_path = os.path.join(output_dir, f"{image_name}.blurred.jpg")

  # Use Detectron2 model to predict location of pitcher
  try:
    d2_pred_output = d2_predictor(image)
    
    # Blur image around pitcher if pitcher was detected
    if d2_pred_output["instances"].pred_classes.cpu().numpy().size > 0:

      # Get location of pitcher in frame
      object_dim = d2_pred_output["instances"].pred_boxes.tensor.cpu().numpy()[0]
      start_point = (int(object_dim[0]), int(object_dim[1]))
      end_point = (int(object_dim[2]), int(object_dim[3]))

      # Blur everything in frame but pitcher
      blurred_img = cv2.blur(image, (75, 75))
      mask = np.zeros((720, 1280, 3), dtype=np.uint8)
      mask = cv2.rectangle(mask, start_point, end_point, (255, 255, 255), -1)
      out_img = np.where(mask==np.array([255, 255, 255]), image, blurred_img)
      cv2.imwrite(out_image_path, out_img)
      return out_image_path

    else:
      pass
  
  except:
    print(f"d2_prediction error with {image}")


def filter_images(images):

  # Find index of first image with pitcher detected
  start_idx = 0
  for i, image in enumerate(images):
    if image != None:
      start_idx = i
      break
    else:
      continue

  # Filter images
  # Only keep image in list if the image is not preceeded by two Nones or more
  none_count = 0
  filtered_images = []
  for i, image in enumerate(images[start_idx:]):

    # If the image is None, do not add to list and add 1 to none_count
    if image == None:
      none_count += 1
      continue

    # If two Nones in a row, stop adding things to the list
    elif none_count >= 2:
      break

    # Otherwise add image to the list
    else:
      filtered_images.append(image)

  # Return paths to filtered images
  return filtered_images


def consolidate_json(json_files):
  '''
  Function to consolidate json files outputted by OpenPose for a sequence of frames
  '''

  # Set objects
  ## 25 body parts and corresponding indices detected and defined by openpose
  keypoint_dict = {
      0:  "Nose",
      1:  "Neck",
      2:  "RShoulder",
      3:  "RElbow",
      4:  "RWrist",
      5:  "LShoulder",
      6:  "LElbow",
      7:  "LWrist",
      8:  "MidHip",
      9:  "RHip",
      10: "RKnee",
      11: "RAnkle",
      12: "LHip",
      13: "LKnee",
      14: "LAnkle",
      15: "REye",
      16: "LEye",
      17: "REar",
      18: "LEar",
      19: "LBigToe",
      20: "LSmallToe",
      21: "LHeel",
      22: "RBigToe",
      23: "RSmallToe",
      24: "RHeel",
      25: "Background"}

  ## Dictionary to store pose data from all frames of the original video
  full_pitch_dict = {
      "Nose":       {"frame": [], "x": [], "y": [], "conf": []},
      "Neck":       {"frame": [], "x": [], "y": [], "conf": []},
      "RShoulder":  {"frame": [], "x": [], "y": [], "conf": []},
      "RElbow":     {"frame": [], "x": [], "y": [], "conf": []},
      "RWrist":     {"frame": [], "x": [], "y": [], "conf": []},
      "LShoulder":  {"frame": [], "x": [], "y": [], "conf": []},
      "LElbow":     {"frame": [], "x": [], "y": [], "conf": []},
      "LWrist":     {"frame": [], "x": [], "y": [], "conf": []},
      "MidHip":     {"frame": [], "x": [], "y": [], "conf": []},
      "RHip":       {"frame": [], "x": [], "y": [], "conf": []},
      "RKnee":      {"frame": [], "x": [], "y": [], "conf": []},
      "RAnkle":     {"frame": [], "x": [], "y": [], "conf": []},
      "LHip":       {"frame": [], "x": [], "y": [], "conf": []},
      "LKnee":      {"frame": [], "x": [], "y": [], "conf": []},
      "LAnkle":     {"frame": [], "x": [], "y": [], "conf": []},
      "REye":       {"frame": [], "x": [], "y": [], "conf": []},
      "LEye":       {"frame": [], "x": [], "y": [], "conf": []},
      "REar":       {"frame": [], "x": [], "y": [], "conf": []},
      "LEar":       {"frame": [], "x": [], "y": [], "conf": []},
      "LBigToe":    {"frame": [], "x": [], "y": [], "conf": []},
      "LSmallToe":  {"frame": [], "x": [], "y": [], "conf": []},
      "LHeel":      {"frame": [], "x": [], "y": [], "conf": []},
      "RBigToe":    {"frame": [], "x": [], "y": [], "conf": []},
      "RSmallToe":  {"frame": [], "x": [], "y": [], "conf": []},
      "RHeel":      {"frame": [], "x": [], "y": [], "conf": []},
      "Background": {"frame": [], "x": [], "y": [], "conf": []}}

  # Internal function
  def chunks(lst, n):
    '''
    Yield successive n-sized chunks from list
    https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
    '''
    for i in range(0, len(lst), n):
      yield lst[i:i + n]

  # Loop through each frame in the pitch
  for json_file in json_files:

    # Disregard frames with no pitcher detected
    if json_file is not None:
      
      # Open json for specific frame
      with open(json_file) as f:
          openpose_keypoints = json.load(f)

      # Get frame number from file name
      frame = json_file.split('.blurred_keypoints.json')[0].split('-')[-1]

      # Split keypoints list into chunks (list of smaller lists) of 3
      # ex: [[x0, y0, conf0], [x1, y1, conf1], ...]
      if openpose_keypoints['people']:
        keypoint_splits = chunks(openpose_keypoints['people'][0]['pose_keypoints_2d'], 3)

      # In the case that OpenPose doesn't detect anyone fill in the frame x, y with 0s
      else:
        keypoint_splits = chunks([0] * 75, 3)

      # Append pose positions for each body part to full_pitch_dict (final json to be stored)
      for i, keypoint in enumerate(keypoint_splits):
        body_position = keypoint_dict[i]
        full_pitch_dict[body_position]["frame"].append(int(frame))
        full_pitch_dict[body_position]["x"].append(keypoint[0])
        full_pitch_dict[body_position]["y"].append(keypoint[1])
        full_pitch_dict[body_position]["conf"].append(keypoint[2])

  return full_pitch_dict


def normalize(vals: list):
    '''
    Normalize each value in a list to between 0 and 1
    x_i = (x_i - x_min) / (x_max - x_min)
    '''
    
    val_min = min(vals)
    val_max = max(vals)
    min_max_range = val_max - val_min

    # Can't divide by 0; Replace min max range with small number if 0
    # Hypothetically the numerator will have to be 0 if min max range is 0
    if min_max_range == 0:
      min_max_range = 0.0001

    return [(val_i - val_min) / min_max_range for val_i in vals]


def empty_files(play_dir):
  subdirs = [os.path.join(play_dir, dir) for dir in os.listdir(play_dir)]
  for subdir in subdirs:
    files_to_empty = [os.path.join(subdir, file_) for file_ in os.listdir(subdir)]
    for file_to_empty in files_to_empty:
      open(file_to_empty, 'w').close()



# Pipeline

In [ ]:
# PIPELINE ---------------------------------------------------------------------
# Get URL and video information
pitcher = "Walker Buehler"
mp4_urls = pitcher_info[pitcher]["video_urls"]
play_ids = pitcher_info[pitcher]["play_ids"]
pitcher_lastname = pitcher.split(" ")[-1].lower()

# Manage primary directories
## Root directory
video2data_root_dir = os.path.join(root_path, "data/video2data")
os.makedirs(video2data_root_dir, exist_ok=True)

## Pitcher specific directory
pitcher_dir = os.path.join(video2data_root_dir, pitcher_lastname)
os.makedirs(pitcher_dir, exist_ok=True)

# Loop through pitches
# download video -> video to images -> blur images -> pose detection -> output data as json
for i, play_id in enumerate(play_ids):

  # Play specific parameters
  print(f"Video {i+1}/{len(play_ids)}...")
  mp4_url = mp4_urls[i]

  # Check if the video has already been processed
  if os.path.exists(f"{os.path.join(pitcher_dir, play_id)}.json"):
    print(f"{os.path.join(pitcher_dir, play_id)}.json already exists. Moving to next play ID...\n")
    continue
  
  # Otherwise continue
  else:

    # Create play specific directory
    play_dir = os.path.join(pitcher_dir, play_id)
    os.makedirs(play_dir, exist_ok=True)

    # Download video
    print(f"Downloading video...")
    video_dir = os.path.join(play_dir, "video")
    video_path = os.path.join(video_dir, mp4_url.split(".com/")[1])
    os.makedirs(play_dir, exist_ok=True)
    download_mp4(video_link=mp4_url, output_dir=video_dir)

    # Video -> images
    print(f"Converting video to images...")
    image_dir = os.path.join(play_dir, "images")
    os.makedirs(image_dir, exist_ok=True)
    mp4_to_image(mp4=video_path, output_dir=image_dir)

    # Blur images
    print(f"Blurring images...")
    blurred_dir = os.path.join(play_dir, "blurred_images")
    os.makedirs(blurred_dir, exist_ok=True)
    blurred_images = sorted(os.listdir(image_dir), key=lambda x: int(x.split("-")[-1].split(".jpg")[0]))
    blurred_images = [os.path.join(image_dir, file_) for file_ in blurred_images]
    blurred_images = [None if img == None else blur_image(img, output_dir=blurred_dir) for img in blurred_images]

    # Filter out non-pitcher images
    print(f"Filtering images...")
    filtered_dir = os.path.join(play_dir, "filtered_images")
    os.makedirs(filtered_dir, exist_ok=True)
    image_to_include = filter_images(blurred_images)
    # Copy images to new directory
    for blurred_image in image_to_include:
      shutil.copy(blurred_image, filtered_dir)
    filtered_images = sorted(os.listdir(filtered_dir), key=lambda x: int(x.split("-")[-1].split(".blurred.jpg")[0]))
    filtered_images = [os.path.join(filtered_dir, file_) for file_ in filtered_images]

    # OpenPose pose detection
    print(f"Running OpenPose...")
    json_dir = os.path.join(play_dir, "json")
    !cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../"$filtered_dir"/ --number_people_max 1 --keypoint_scale 0 --write_json ../"$json_dir"/ --render_pose 0 --display 0 #--write_video ../"$play_dir"/ --write_video_fps 25
    json_files = [img.replace('filtered_images', 'json').replace('.jpg', '_keypoints.json') for img in filtered_images]

    # Consolidate frame-specific json files into one json file
    pitch_json = consolidate_json(json_files)

    # Output json
    with open(os.path.join(pitcher_dir, f"{play_id}.json"), 'w') as f:
        json.dump(pitch_json, f)
    print(f"JSON outputted successfully...")

    # Shortcut to empty all data from temporary files before sending to trash
    #print(f"Emptying intermediate files in {play_dir}...")
    #empty_files(play_dir)

    # Remove temporary files
    print(f"Deleting files now...")
    !rm -rf "$play_dir"
    print("Done.\n")


Video 1/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/4d2c3d49-94b9-4b51-a77a-b4504c963f42.json already exists. Moving to next play ID...

Video 2/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/5b639d2c-4d72-4ea5-9986-40ab2cbcd4e8.json already exists. Moving to next play ID...

Video 3/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/f30b5fb1-a47a-4b9a-953a-a9e291c25adc.json already exists. Moving to next play ID...

Video 4/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/c6ebe342-c1b9-4269-b8bd-ed13ede641df.json already exists. Moving to next play ID...

Video 5/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/10899a22-aaf1-4a26-9c1e-b8ed6b5c0420.json already exists. Moving to next play ID...

Video 6/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/dd49f6ba-9c3a-41b9-813d-d7506a22c006.json already exists. Moving to next play ID...

Video 7/1164...
drive/MyDrive/NNextPitch/data/video2data/buehler/cf490adb-1064-44ad-8ef4-b41cd81fe8f